## Loading Data Set

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
## IMPORT DEPENDENCIES

from bs4 import BeautifulSoup as bs
import gzip
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from collections import OrderedDict
import sqlite3
import ipynb.fs
print ("loading data set dependencies successful")

loading data set dependencies successful


In [3]:
## SET FILE META VARIABLES

corpus_path = "/nfs/trects-kba2014-filtered" # directory of corpus of gzipped html files
topics_path = corpus_path + "/test-topics.xml"
doc_tags = ['topic_id','streamid', 'docid', 'yyyymmddhh', 'kbastream', 'zulu', 'epoch', 'title', 'text', 'url'] # doc fields
topic_tags = ['id', 'title', 'description', 'start','end','query','type'] # topic fields
test_file_addr = corpus_path + "/1/2012-02-22-15.gz"
# database address variables
db_dir = '/nfs/proj-repo/AAARG-dissertation'
db_name = 'sumresults.db'
db_path = db_dir + '/' + db_name

In [4]:
# open and get beautifulsoup object from markup file
def open_markup_file(addr, gz=True, xml=False, verbose=False):
    markup = None
    f = None
    
    if verbose:
        print(addr)

    if gz:
        f = gzip.open(addr)
    else:
        f = open(addr)
        
    if xml == False:
        markup = bs(f)  # open as html
    else:
        markup = bs(f, "xml")
        
    f.close()
    return markup


# parse markup and return 2D list [entry:tags]
def parse_markup(markup, entry_list, find_tag="doc", tag_list=doc_tags, topic_id=None):
    for e in markup.find_all(find_tag):
        entry = OrderedDict.fromkeys(tag_list)
        if topic_id is not None:
            entry['topic_id'] = topic_id
        for c in e.children:  # children use direct children, descendants uses all
            if c.name in entry:
                entry[c.name] = str(c.string)
            elif c.name is None and c.string != '\n':  # inner body of <doc> tag
                entry['text'] = str(c.string)
        entry_list.append(list(entry.values()))
        
            
# recursively find gz html files from a directory address
def search_dir(path):    
    # separate the subdirectories and html files 
    # (help maintain sequential order of insertion)
    gz_paths = []
    for f in os.scandir(path):
        if os.path.splitext(f.path)[-1].lower() == ".gz":
            gz_paths.append(f.path)
    
    return gz_paths


def list_to_dataframe(markup_list, tags):
    return pd.DataFrame(markup_list, columns=tags)

In [5]:
# load topics into dataframe
def load_topics(path):
    topics_list = []
    
    parse_markup(open_markup_file(path, gz=False, xml=True), 
                    topics_list, find_tag="event", tag_list=topic_tags)
    
    df = list_to_dataframe(topics_list, topic_tags)
    df['id'] = pd.to_numeric(df['id'])
    return df

topics = load_topics(topics_path)

In [6]:
print("Topics loaded successfuly")
print(display(topics[0:4]))

Topics loaded successfuly


,id,title,description,start,end,query,type
0,1,2012 Buenos Aires Rail Disaster,http://en.wikipedia.org/wiki/2012_Buenos_Aires...,1329910380,1330774380,buenos aires train crash,accident
1,2,2012 Pakistan garment factory fires,http://en.wikipedia.org/wiki/2012_Pakistan_gar...,1347368400,1348232400,pakistan factory fire,accident
2,3,2012 Aurora shooting,http://en.wikipedia.org/wiki/2012_Aurora_shooting,1342766280,1343630280,colorado shooting,shooting
3,4,Wisconsin Sikh temple shooting,http://en.wikipedia.org/wiki/Wisconsin_Sikh_te...,1344180300,1345044300,sikh temple shooting,shooting


None


### Loading Topics Into Database

In [44]:
from .defs.database_management import create_tables, populate_topics  # import database_management functions
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
create_tables(conn, cursor,path=db_path)
populate_topics(conn, cursor, db_path, topics)
conn.close()

creating topic table...
creating techniques table...
creating instances table...
creating meta table...
creating updates table...
creating nuggets table...
creating linking nuggets table
committing changes
committed
added topics_df


ipynb.fs.defs.database_management:134: UserWarning: The topics table already has entries


## Load Corpus

In [8]:
# load all formatted gzipped html files into dataframe
def load_corpus(path):
    #corpus_list = []
    df = pd.DataFrame(columns=doc_tags)
    for topic_id in topics['id'].to_numpy():
        print("Loading topic " + str(topic_id) + "...")
        topic_list = []
        id_path = corpus_path + "/" + str(topic_id) + "/"  # every topic id correlates to subfolder named after it
        gz_paths = search_dir(id_path)
        for gz_path in tqdm(gz_paths, position=0, leave=True):
            parse_markup(open_markup_file(gz_path, verbose=False),
                            topic_list, topic_id=topic_id)
        topic_df = list_to_dataframe(topic_list, doc_tags)
        df = df.append(topic_df)
    df['epoch'] = pd.to_numeric(df['epoch'])
    return df

corpus = load_corpus(corpus_path)
print("Corpus loaded Successfully")

Loading topic 1...


  0%|          | 0/241 [00:00<?, ?it/s]

Loading topic 2...


  0%|          | 0/241 [00:00<?, ?it/s]

Loading topic 3...


...

Loading topic 9...


  0%|          | 0/241 [00:00<?, ?it/s]

Loading topic 10...


100%|██████████| 241/241 [01:21<00:00,  2.96it/s]

Corpus loaded Successfully


In [9]:
print("Corpus loaded succesfully: " + str(len(corpus)) + " documents loaded.")
print(display(corpus[0:4]))
# there is an error in the dataset that article at 1 is misplaced in topic 1

Corpus loaded succesfully: 12261 documents loaded.


,topic_id,streamid,docid,yyyymmddhh,kbastream,zulu,epoch,title,text,url
0,1,1330269540-995ed81eafa60498872335da7dce1386,995ed81eafa60498872335da7dce1386,2012-02-26-15,news,2012-02-26T15:19:00.000000Z,1330269540,US says it's steadfast in rebuilding Afghanist...,\nUS says it's steadfast in rebuilding Afghani...,http://www.elpasotimes.com/politics/ci_20049216
1,1,1330268520-f42a863b58b2cc53cc716953c40f6065,f42a863b58b2cc53cc716953c40f6065,2012-02-26-15,news,2012-02-26T15:02:00.000000Z,1330268520,Argentina Train Crash: Driver Blames Faulty Br...,\nArgentina Train Crash: Driver Blames Faulty ...,http://www.thisdaylive.com/articles/argentina-...
2,1,1330270020-e47e013ec518f5fdd253ce28231f509f,e47e013ec518f5fdd253ce28231f509f,2012-02-26-15,news,2012-02-26T15:27:00.000000Z,1330270020,The Alaska Journal of Commerce Local News Oil ...,\nThe Alaska Journal of Commerce Local News Oi...,http://ap.alaskajournal.com/pstories/20120226/...
3,1,1330268700-8078290575c82c8dd0e4e99370447bd2,8078290575c82c8dd0e4e99370447bd2,2012-02-26-15,news,2012-02-26T15:05:00.000000Z,1330268700,U.S. military receives remains of last soldier...,\nU.S. military receives remains of last soldi...,http://www.islandpacket.com/2012/02/26/1978117...


None


In [10]:
# # duplicates are updates to the page
# find_nug = corpus[corpus['streamid'] == "1329915660-47ed792a77d798dda8697654e8fcbb43"]
# # 1329915300-46c9b2db03fbaf7d2a903bbfa7ff3c93-3
# # duplicate found when -3 taken away
# dup_nug = corpus[corpus['streamid'] == "1329915300-46c9b2db03fbaf7d2a903bbfa7ff3c93"]
# print(corpus[corpus['docid'] == "47ed792a77d798dda8697654e8fcbb43"])
# print(find_nug)
# print(dup_nug)
# # print(dup_nug['text'])

## Preprocessing

- Topic Modelling needs better preprocessing (stop words/lemmas etc.)
    - stop words
    - lemmatization (stemming is faster but is rule-based with more false transformations)
    - special char removal
- Could try removing junk at top of docs through REs/spacy

In [11]:
# ## IMPORT DEPENDENCIES

# import spacy

# print("preprocessing dependencies import successful")

preprocessing dependencies import successful


In [12]:
# nlp = spacy.load("en_core_web_sm")  # try experimenting disabling parts of spacy pipeline see if .sents still works

# nlp.add_pipe(nlp.create_pipe('sentencizer'), before="parser")
# nlp.remove_pipe('tagger')
# nlp.remove_pipe('parser')

('parser', <spacy.pipeline.pipes.DependencyParser at 0x7f410b5435e8>)

In [13]:
# test_docs = corpus.loc[0:3,:]  # work on just top 3 for now

# # in data frame, split sentences into list by the newline delimiter
# #test_docs['text'] = test_docs['text'].map(lambda x: x.splitlines())

# # map the non-preprocessed string list to a preprocessed string list

# #@Tokenize
# def spacy_tokenize(string):
#     tokens = list()
#     doc = nlp(string)
#     for token in doc:
#         if not token.is_stop:
#             tokens.append(token)
#     return tokens

# #@Normalize
# def normalize(tokens):
#     normalized_tokens = list()
#     for token in tokens:
#         normalized = token.text.lower().strip()
#         if ((token.is_alpha or token.is_digit)):
#             normalized_tokens.append(normalized)
#     return normalized_tokens

# #@Tokenize and normalize
# def tokenize_normalize(string):
#     return normalize(spacy_tokenize(string))

# # test_prep = []
# # for doc in test_docs['text']:
# #     d = []
# #     for sent in doc:
# #         d.append(tokenize_normalize(sent))
# #     test_prep.append(d)
        
# print("cell finished")

In [14]:
#print(test_prep[0])

## Retrieve Word and Sentence Level Embeddings

In [15]:
# from sentence_transformers import SentenceTransformer
# from transformers import AutoModel, AutoTokenizer
# #sent_model = AutoModel.from_pretrained('sentence-transformers/distilbert-base-nli-stsb-mean-tokens')
# sent_tokenizer = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
# #word_model = AutoModel.from_pretrained('distilbert-base-uncased')
# word_tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

## Topic Modelling

- LDA uses K-means clustering
- HDA learns num topics automatically (Bayesian non-parametric)

In [16]:
# # word level topic modelling
# # needs better preprocessing (remove stopwords/lemmitization etc)
# # maybe add REs/other preprocessing remove uninformative junk at top of docs

# import gensim
# from gensim import corpora

# print("loaded dependencies")

loaded dependencies


In [17]:
# class TopicModeller: 
#     def __init__(self):
#         self.model = None
#         self.corpus_dict = None
#         self.weighted_tokens = None
#         self.print_topics = None
        
#     def weigh_tokens(self, texts, method="bow"):
#         """Perform token weighting scheme on text and return with dict"""
#         def create_dictionary(texts):
#             """Create a gensim dictionary of index-word mappings"""
#             return corpora.Dictionary(texts)
    
#         flat_texts = [token for sent in texts for token in sent]  # should be fast
#         self.corpus_dict = create_dictionary(flat_texts)
#         if method == "bow":
#             self.weighted_tokens = [self.corpus_dict.doc2bow(text) for text in flat_texts]
#         else:
#             raise Exception("Incorrect method parameter")
            
#     def model_topics(self, method="lda", num_topics=10):
#         if method == "lda":
#     #         model = gensim.models.ldamodel.LdaModel(weighted_tokens, num_topics=NUM_TOPICS, 
#     #                                                 id2word=corpus_dict, passes=15)
#             self.model = gensim.models.ldamulticore.LdaMulticore(self.weighted_tokens, num_topics=num_topics, 
#                                                     id2word=self.corpus_dict, passes=15)
#         else:
#             raise Exception("Incorrect method parameter")
            
#         self.print_topics = self.model.print_topics()
#         return self.print_topics
# print("loaded cell")

loaded cell


In [18]:
# topic_model = TopicModeller()
# topic_model.weigh_tokens(test_prep)
# print(topic_model.model_topics())

## Sentence Level Clustering

In [19]:
# # first get sentences which are nearest neighbors to the identified topics
# # https://scikit-learn.org/stable/modules/neighbors.html
# # https://stackoverflow.com/questions/60996584/bert-embedding-for-semantic-similarity
# # https://stackoverflow.com/questions/59865719/how-to-find-the-closest-word-to-a-vector-using-bert
# # https://gist.github.com/avidale/c6b19687d333655da483421880441950


# # then compare sentence results from pure extractive summariser maybe?

# from sklearn.neighbors import KDTree
# #import mxnet as mx
# from bert_embedding import BertEmbedding

# # ctx = mx.gpu(0)
# # bert = BertEmbedding(ctx=ctx)
# bert_emb = BertEmbedding()

ModuleNotFoundError: No module named 'bert_embedding'

In [ ]:
# """
# Trying code from here
# https://gist.github.com/avidale/c6b19687d333655da483421880441950

# Preprocess embeddings in a formatted way as such can track sentences, words, embeddings

# do this, then pass the LDA topics into the query
# """ 

# class EmbeddingHandler:
#     def __init__(self, sentences, model):
#         self.sentences = sentences
#         self.model = model
        
#     def generate_embeddings(self):
#         result = self.model(self.sentences)
# #         result = list()
# #         for sent in self.sentences:
# #             result.append(self.model.encode(sent, is_pretokenized=True))
# #         #result = self.model.encode(self.sentences, is_pretokenized=True, show_progress_bar=True)
#         #print(result)
        
#         self.sent_ids = []
#         self.token_ids = []
#         self.tokens = []
#         embeddings = []
#         for i, (toks, embs) in enumerate(tqdm(result)):
#             for j, (tok, emb) in enumerate(zip(toks, embs)):
#                 self.sent_ids.append(i)
#                 self.token_ids.append(j)
#                 self.tokens.append(tok)
#                 embeddings.append(emb)
#         embeddings = np.stack(embeddings)
#         # we normalize embeddings, so that euclidian distance is equivalent to cosine distance
#         self.normed_embeddings = (embeddings.T / (embeddings**2).sum(axis=1) ** 0.5).T
        
#     def generate_sent_embeddings(self):
#         """test sent vs word embeddings"""
#         # use sentence-transformers embeddings
#         result = self.model.encode(self.sentences)
#         self.sent_ids = []
#         self.tokens = []
#         embeddings = []
#         for i, (tok, emb) in enumerate(tqdm(zip(self.sentences,result))):
#             self.sent_ids.append(i)
#             self.tokens.append(tok)
#             embeddings.append(emb)
#         embeddings = np.stack(embeddings)
#         # we normalize embeddings, so that euclidian distance is equivalent to cosine distance
#         self.normed_embeddings = (embeddings.T / (embeddings**2).sum(axis=1) ** 0.5).T
        
#     def create_comparitor(self):
#         # this takes some time
#         self.indexer = KDTree(self.normed_embeddings)
#         print("created KDTree")
    
#     def query(self, query_sent, query_word, k=10, filter_same_word=False):
#         toks, embs = self.model([query_sent])[0]

#         found = False
#         for tok, emb in zip(toks, embs):
#             if tok == query_word:
#                 found = True
#                 break
#         if not found:
#             raise ValueError('The query word {} is not a single token in sentence {}'.format(query_word, toks))
#         emb = emb / sum(emb**2)**0.5

#         if filter_same_word:
#             initial_k = max(k, 100)
#         else:
#             initial_k = k
#         di, idx = self.indexer.query(emb.reshape(1, -1), k=initial_k)  # this is returning our neighbours
#         distances = []
#         neighbors = []
#         contexts = []
#         # this is filtering for word matching
#         for i, index in enumerate(idx.ravel()):
#             token = self.tokens[index]
#             if filter_same_word and (query_word in token or token in query_word):  # take this away
#                 continue
#             distances.append(di.ravel()[i])
#             neighbors.append(token)
#             contexts.append(self.sentences[self.sent_ids[index]])
#             if len(distances) == k:
#                 break
#         return distances, neighbors, contexts
    
#     def topic_neighbors(self, topic_word, k=10):
#         # get average embedding of topic word
#         # maybe instead return context sentence that is closest to averaged embedding?
#         # that way can use context to get right meaning
#         topic_emb = self.avg_embedding(self.retrieve_embeddings(topic_word))
        
#         # get neighbors
#         # do I need reshape?
#         di, idx = self.indexer.query(topic_emb.reshape(1,-1), k=k)
#         distances = []
#         neighbors = []
#         contexts = []
#         for i, index in enumerate(idx.ravel()):
#             token = self.tokens[index]
#             distances.append(di.ravel()[i])
#             neighbors.append(token)
#             contexts.append(self.sentences[self.sent_ids[index]])
#         return distances, neighbors, contexts
        
        
#     def retrieve_embeddings(self, token):
#         idxs = []
#         for i, t in enumerate(self.tokens):
#             if t == token:
#                 idxs.append(i)
#             elif token in t:  # sent-embeddings temp workaround
#                 idxs.append(i)
#         embs = []
#         for i in idxs:
#             embs.append(self.normed_embeddings[i])
#         return embs
    
#     def avg_embedding(self, emb_list):
#         return np.mean(emb_list, axis=0)

In [20]:
# emb_handler = EmbeddingHandler(emb_handler_corp, bert_emb)  # [0] index taking first doco
# emb_handler.generate_embeddings()
# emb_handler.create_comparitor()

## Summarization

In [21]:
from summarizer import Summarizer
#from summarizer.coreference_handler import CoreferenceHandler
#co_handler = CoreferenceHandler(greedyness=0.4)
#sum_model = Summarizer(sentence_handler=co_handler)
sum_model = Summarizer()
print("loaded summarisation model")

loaded summarisation model


In [22]:
# for processing massive strings
#!jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

In [23]:
"""
ValueError: [E088] Text of length 3496277 exceeds maximum of 1000000. 
The v2.x parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. 
This means long texts may cause memory allocation errors. 
If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. 
The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.
"""

class SummarizationHandler:
    def __init__(self, model):
        self.model = model
        
    def summarize(self, texts, max_len=500000, smallest_ratio=0.1):
        def sum_texts(texts_list, ratio=None):
            sums = []
            print("Summarising text in " + str(len(texts_list)) + " pieces.")
            for text in tqdm(texts_list):
                if ratio is None:
                    sums.append(self.model(text))
                else:
                    sums.append(self.model(text, ratio=ratio))
            return ". ".join(sums)
        
        def half_list(a_list):
            half = len(a_list)//2
            return a_list[:half], a_list[half:]
        
        total_len = self.total_length(texts)
        split_texts, size_split = self.split_texts(texts, total_len, max_len)
        split_ratio = size_split / total_len
        
        new_splits = [split_texts]
        new_ratios = [split_ratio]
        while split_ratio < smallest_ratio:
            new_ratios = []
            n_s = []
            for split in new_splits:  # loop for each list:
                t1, t2 = half_list(split)
                n_s.append(t1)
                n_s.append(t2)
            new_splits = n_s
            for s in new_splits:
                new_ratios.append(size_split/self.total_length(s))
            split_ratio = min(new_ratios)
        
        split_sums = []
        if new_ratios[0] <= 1:
            for i in range(len(new_splits)):
                split_sum = sum_texts(new_splits[i], ratio=new_ratios[i])
                split_sums.append(split_sum)
            s = ". ".join(split_sums)
            s = sum_texts([s])
            return s
        else:
            return sum_texts(new_splits[0])
        
#         if split_ratio >= 1:
#             cur_sum = sum_texts(split_texts)
#         else:
#             sums = sum_texts(split_texts, split_ratio)
#             cur_sum = ". ".join(sums)
#             cur_sum = sum_texts([cur_sum])
#         return cur_sum
        
    def split_texts(self, texts, total_len, max_len):
        """Split texts into list of strings under max_len"""
        num_split, size_split = self.optimal_split(total_len, max_len)
        splits = []
        cur_split = ""
        i = 1
        for text in texts:
            if i == num_split:
                # just add to last no check
                cur_split += text
            else:
                if (len(cur_split) + len(text) > size_split):
                    splits.append(cur_split)
                    cur_split = text
                    i += 1
                else:
                    cur_split += text
        splits.append(cur_split)
        return splits, size_split
            
    def optimal_split(self, total_len, max_len):
        """Find even split of text under max_len"""
        under_len = int(max_len * 0.95)  # use slightly under max_len for safety
        cur_div = 1
        cur_size = total_len / cur_div
        while (cur_size > under_len):
            cur_div += 1
            cur_size = total_len / cur_div
        return cur_div, cur_size
        
    
    def total_length(self,texts):
        total = 0
        for t in texts:
            total += len(t)
        return total



In [24]:
sum_handler = SummarizationHandler(sum_model)

test_topic = corpus.loc[corpus['topic_id'] == 1]['text'][0:10]
test_topic_sum = sum_handler.summarize(test_topic)

print(test_topic_sum)
print("complete")

  0%|          | 0/1 [00:00<?, ?it/s]

Summarising text in 1 pieces.


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:972: ConvergenceWarning: Number of distinct clusters (624) found smaller than n_clusters (662). Possibly due to duplicate points in X.
  return_n_iter=True)
  0%|          | 0/1 [00:00<?, ?it/s]

Summarising text in 1 pieces.


100%|██████████| 1/1 [00:42<00:00, 42.37s/it]

US says it's steadfast in rebuilding Afghanistan - El Paso Times Mobile e-Edition Today’s Print Ads Newsletters Customer Service Subscribe This Site Web Search powered by YAHOO ! We've got to create a situation in which al-Qaida is not coming back." The Pentagon on Sunday identified Air Force Lt. It's an extraordinary admission of failure for us to establish the relationships that you'd have to have for a successful transition to the Afghan military and Afghan security leadership," Romney said. So they are very much in this fight trying to protect us." This material may not be published, broadcast, rewritten or redistributed. Investigators said they would check the control room recordings. Driver, Marcos Cordoba is being investigated on suspicion of what Argentine law calls "guilty damage without an attempt to cause harm". Blood tests showed Cordoba had not been drinking or using drugs when the accident happened. Got ’ Read more... 14 Apr 2012 In Praise of a Good Read Read more... 14 A

## Evaluation Metrics

### Reading 'Nugget' Values

In [25]:
nugget_dir = "/nfs/TemporalSummarization/ts13/results"
updates_sampled_path = nugget_dir + "/updates_sampled.tsv"
nuggets_path = nugget_dir + "/nuggets.tsv"
nug_matches_path = nugget_dir + "/matches.tsv"

In [26]:
import re

def find_duplicates(df):
    seen = set()
    seen_twice = set()
    for docid in df['docid']:
        if docid not in seen:
            seen.add(docid)
        else:
            seen_twice.add(docid)
    return seen_twice

def create_update_df():
    """Data Frame containing information about docs which have updates/multiple instances in corpus"""
    def create_entry(row, col_tags):
        entry = {}
        for col in col_tags:
            entry[col] = row[col]
        return entry
    
    col_tags = ['docid', 'streamid', 'epoch', 'yyyymmddhh', 'zulu']
    entry_list = []
    dups = find_duplicates(corpus)
    for docid in tqdm(dups, position=0, leave=True):
        d = corpus[corpus['docid'] == docid]
        for index, row in d.iterrows():
            entry = create_entry(row, col_tags)
            entry_list.append(entry)
             
    update_df = pd.DataFrame(entry_list)
    update_df = update_df.set_index(col_tags)
    return update_df
                    
                
def create_nugget_df():
    """Dataframe containing nugget data and its appearances in corpus"""
    def create_entry(row, reg_cols, multi_col_vals=None):
        entry_dict = {}
        for col in reg_cols:
            entry_dict[col] = row[col]
        if multi_cols is not None:
            for k,v in multi_col_vals.items():
                entry_dict[k] = v
        return entry_dict
        
    nuggets_tsv = pd.read_csv(nuggets_path, "\t")
    entry_list = []
    reg_cols = ['query_id', 'nugget_id', 'importance', 'nugget_len', 'nugget_text']
    multi_cols = ['docid', 'streamid', 'epoch', 'yyyymmddhh']  # multiindex cols
    num_cols = ['query_id', 'importance', 'nugget_len', 'epoch']
    
    pbar = tqdm(total=len(nuggets_tsv), position=0, leave=True)
    for index, row in nuggets_tsv.iterrows():
        # find where nugget appears in text
        nug_text = row['nugget_text']
        topic_id = 0
        try:
            topic_id = int(row['query_id'])  # make sure pattern match in correct topic
        except ValueError:
            pbar.update()
            continue  # topic_id is unknown string in tsv file, e.g. "TS13.07"
        appears = corpus[corpus['topic_id'] == topic_id]
        appears = appears[appears['text'].str.contains(re.escape(nug_text))]  # make sure no accidental regex pattern
        
        # gather information on docs it appears in
        dups = find_duplicates(appears)  # get docids where nugget appears
        for docid in dups:
            upd = appears[appears['docid'] == docid]  # get docs with this docid
            for i, r in upd.iterrows():  # gather info on each doc with this docid (e.g. streamid, epoch etc.)
                multi_col_vals = {}
                for multi_col in multi_cols:
                    multi_col_vals[multi_col] = r[multi_col]
                entry = create_entry(row, reg_cols, multi_col_vals=multi_col_vals)
                entry_list.append(entry)
        pbar.update()
    pbar.close()
    
    # form multi-index nugget dataframe
    reg_cols.extend(multi_cols)  # get new multiindex order
    nugget_df = pd.DataFrame(entry_list)
    nugget_df[num_cols] = nugget_df[num_cols].apply(pd.to_numeric, errors='coerce', axis=1)  # convert appropriate cols to numerical values
    nugget_df.rename(columns={'query_id':'topic_id'}, inplace=True)  # topic_id matches other dataframes
    return nugget_df

In [27]:
nugget_df = create_nugget_df()

100%|██████████| 1366/1366 [00:41<00:00, 33.29it/s] 


In [28]:
display(nugget_df[0:5])

,topic_id,nugget_id,importance,nugget_len,nugget_text,docid,streamid,epoch,yyyymmddhh
0,1,VMTS13.01.052,3,2,Hundreds injured,dd95d5dbbff443c3ddae4e34a5d2e9c1,1330041420-dd95d5dbbff443c3ddae4e34a5d2e9c1,1330041420,2012-02-23-23
1,1,VMTS13.01.052,3,2,Hundreds injured,dd95d5dbbff443c3ddae4e34a5d2e9c1,1330041420-dd95d5dbbff443c3ddae4e34a5d2e9c1,1330041420,2012-02-23-23
2,1,VMTS13.01.054,1,3,"February 22, 2012",f66f6668504592a391345e012800469c,1329944400-f66f6668504592a391345e012800469c,1329944400,2012-02-22-21
3,1,VMTS13.01.054,1,3,"February 22, 2012",f66f6668504592a391345e012800469c,1329944400-f66f6668504592a391345e012800469c,1329944400,2012-02-22-21
4,1,VMTS13.01.054,1,3,"February 22, 2012",ecda22bcfc10da137b49f0089bd5d7f5,1329916140-ecda22bcfc10da137b49f0089bd5d7f5,1329916140,2012-02-22-13


In [29]:
update_df = create_update_df()

100%|██████████| 1245/1245 [00:02<00:00, 594.41it/s]


In [30]:
display(update_df[0:5])

Empty DataFrame
Columns: []
Index: [(29874677aff307363157298b13c8becf, 1354014393-29874677aff307363157298b13c8becf, 1354014393, 2012-11-27-11, 2012-11-27T11:06:33.0Z), (29874677aff307363157298b13c8becf, 1354014284-29874677aff307363157298b13c8becf, 1354014284, 2012-11-27-11, 2012-11-27T11:04:44.0Z), (29874677aff307363157298b13c8becf, 1354014284-29874677aff307363157298b13c8becf, 1354014284, 2012-11-27-11, 2012-11-27T11:04:44.0Z), (29874677aff307363157298b13c8becf, 1354014292-29874677aff307363157298b13c8becf, 1354014292, 2012-11-27-11, 2012-11-27T11:04:52.0Z), (29874677aff307363157298b13c8becf, 1354014260-29874677aff307363157298b13c8becf, 1354014260, 2012-11-27-11, 2012-11-27T11:04:20.0Z)]

In [31]:
# select random selection of streams with nugs to summarise
test_nug = nugget_df[nugget_df['topic_id'] == 1].sample(n=10)['streamid']
test_nug = corpus[corpus['streamid'].isin(test_nug)]
test_nug_sum = sum_handler.summarize(test_nug['text'])
print(test_nug_sum)

  0%|          | 0/1 [00:00<?, ?it/s]

Summarising text in 1 pieces.


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:972: ConvergenceWarning: Number of distinct clusters (137) found smaller than n_clusters (269). Possibly due to duplicate points in X.
  return_n_iter=True)
  0%|          | 0/1 [00:00<?, ?it/s]

Summarising text in 1 pieces.


100%|██████████| 1/1 [00:11<00:00, 11.42s/it]

2 'Cuse, reaches Big East final Woman missing for a month found with her cat 5 Irish killed in 1832 re-buried in Pa. cemetery Teen smoking an 'epidemic,' new report finds Pakistan charges Osama bin Laden 's widows Anti- Obama Facebook post tests military rules Anna Gristina: Suburban mom or high-rolling madam? The anger over the Quran burning has sparked two days of protests across Afghanistan and tapped into anti-foreign sentiment fueled by a popular perception that foreign troops disrespect Afghan culture and Islam . The Afghan Interior Ministry said in a statement that clashes during a protest in the eastern province of Parwan left four people dead. It said an investigation was under way to determine what happened. U.S.
Gen.
John Allen , the top commander of American and NATO forces in Afghanistan , said after the books had been mistakenly given to troops to be burned at a garbage pit without realizing it. " The moment we found out about it we immediately stopped and we intervened."

## Evaluation Metrics

In [32]:
from rouge_score import rouge_scorer

In [33]:
class MetricHandler:
    def __init__(self, rouge_scores=['rouge1']):   
#         self.summary = summary
        self.rouge = rouge_scorer.RougeScorer(rouge_scores)
        self.nugget_df = None
        self.nugget_dict = None
        self.streamids = None
        
    def evaluate_summary(self, summary, streamids, rouge=True, importance=True):
        self.streamids = streamids
        self.nugget_df = self.nugget_frame(streamids)  # store in self.nugget_frame
        # create nested dictionary of metrics
        metrics = {}
        if importance:
            sum_nugs = self.nuggets_in_summary(summary)
            cur_imp, total_imp = self.importance_score(sum_nugs)
            imp_dict = {}
            imp_dict['cur_imp'] = cur_imp
            imp_dict['total_imp'] = total_imp
            metrics['importance'] = imp_dict
        if rouge:
            target_text = self.target_nugget_text()
            rouges = self.rouge_score(target_text, summary)
            for k,v in rouges.items():
                r_dict = {}
                for label, value in v._asdict().items():
                    # keys: precision, recall, fmeasure
                    r_dict[label] = value
                metrics[k] = r_dict
        return metrics, sum_nugs  # return found nuggets to pass to db
        
    def rouge_score(self, target, summary):
        scores = self.rouge.score(target, summary)
        return scores
        
    def target_nugget_text(self, str_divider=" "):
        t_nugs = list(self.nugget_df['nugget_text'])
        t_nugs = str_divider.join(t_nugs)
        return t_nugs
        
    def importance_score(self, sum_nugs):
        cur_score = sum_nugs['importance'].sum()  # actual summary score
        total_score = self.nugget_df['importance'].sum() # potential score
        return cur_score, total_score
        
    def nuggets_in_summary(self, summary):
        # filter where nugget_text is in summary
        sum_nugs = self.nugget_df[self.nugget_df.apply(lambda x: x['nugget_text'] in summary, axis=1)]
        return sum_nugs
        
    def nugget_frame(self, streamids, keep_columns=None):
        if keep_columns is None:
            keep_columns = ['nugget_id', 'importance', 'nugget_text']
        # get nuggets for each streamid
        nug_rows = nugget_df[nugget_df['streamid'].isin(streamids)]
        nug_rows = nug_rows.drop_duplicates('nugget_id')
        nug_rows = nug_rows[keep_columns]
        self.nugget_df = nug_rows
        return self.nugget_df
    
    def update_summary(self, summary):
        self.summary = summary

In [34]:
metric_handler = MetricHandler()
metric_handler.evaluate_summary(test_nug_sum, test_nug['streamid'])

({'importance': {'cur_imp': 1, 'total_imp': 12},
  'rouge1': {'precision': 0.004761904761904762,
   'recall': 0.38461538461538464,
   'fmeasure': 0.009407337723424272}},
        nugget_id  importance        nugget_text
 2  VMTS13.01.054           1  February 22, 2012)

In [35]:
# # find difference in epoch for a day
# def epoch_diff():
#     # find instances in epoch where there is a day gap
#     day_gap['']

## Summarisation Driver

In [49]:
from datetime import datetime

class SummaryFeeder:
    """
    Need to add nugget metric info to database table and add from here
    Then also add to nugget_instances table
    """
    def __init__(self, sum_handler, tech_name, tech_descr=None, is_temporal=False):
        self.sum_handler = sum_handler
        self.tech_name = tech_name
        self.is_temporal = is_temporal
        self.tech_descr = tech_descr
        self.metric = MetricHandler()
        
    def summarize_topics(self, corp_df):
        # pre-determine some columns to be inputted into meta table
        self.meta_columns = self.get_meta_columns(corp_df)
        # pre-summary database operations
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()
        
        start_exec = self.cur_datetime()
        instance_id = self.store_instance(start_exec)
        self.conn.commit()
        
        topic_ids = corp_df['topic_id'].unique()
        for topic_id in tqdm(topic_ids, position=0, leave=True):
            # currently sum all 
            topic_df = corp_df[corp_df['topic_id'] == topic_id]
            summary = self.sum_handler.summarize(topic_df['text'])
            self.store_topic_summary(topic_id, instance_id, summary)
            self.conn.commit()
        
        end_exec = self.cur_datetime()
        self.update_end_exec(end_exec, instance_id)
        self.conn.commit()
        self.conn.close()
        print("summarize_topics complete")
        
    def get_meta_columns(self, corp_df):
        meta_fields = {}
        for topic_id in corp_df['topic_id'].unique():
            nest = {}
            t = corp_df[corp_df['topic_id'] == topic_id]
            t = t.sort_values("epoch")
            nest['epoch_start'] = t['epoch'].iloc[0]
            nest['epoch_end'] = t['epoch'].iloc[-1]
            streamids = list(t['streamid'])
            streamids = ",".join(streamids)
            nest['streamids'] = streamids
            meta_fields[topic_id] = nest
        return meta_fields
            
    def store_instance(self, start_exec):
        # get number of rows to get instance value
        self.cursor.execute('SELECT COUNT(instance) FROM instances')
        rowcount = self.cursor.fetchone()[0]
        
        # insert instance
        self.cursor.execute('insert into instances values(?,?,?,?,?)', 
                  (rowcount, self.tech_name, self.is_temporal, start_exec, None))  # add end_exec later
        return rowcount # return instance id for ease of later storage
    
    def update_end_exec(self, end_exec, instance_id):
        self.cursor.execute('UPDATE instances SET end_exec=? WHERE instance=?', (end_exec, instance_id))
        
            
    def store_topic_summary(self, topic_id, instance_id, summary):
        self.cursor.execute('insert into meta values(?,?,?,?,?,?,?)',
                  (topic_id, instance_id, summary, self.meta_columns[topic_id]['streamids'],
                   self.meta_columns[topic_id]['epoch_start'], self.meta_columns[topic_id]['epoch_end'], self.is_temporal))
        
    def fetch_technique_entry(self):
        self.cursor.execute('SELECT * FROM techniques WHERE name=?', (self.tech_name,))
        entry = self.cursor.fetchone()
        return entry
        
    def insert_technique(self, tech_descr=None):
        entry = self.fetch_technique_entry()
        if entry is None:  # technique not in database
            if tech_descr is None:
                raise ValueError("Tech description must not equal none if technique not in database")
            else:
                self.cursor.execute('insert into techniques values (?, ?)', (self.tech_name, tech_descr))
                return True
        else:
            return False
        
    def cur_datetime(self):
        time = datetime.now().strftime("%B %d, %Y %I:%M%p")
        return time

In [50]:
# Full corpus caused float divison by zero error at new_ratios.append(size_split/self.total_length(s))

def small_corpus(start_topic, end_topic):
    t_dfs = []
    topic_ids = corpus['topic_id'].unique()
    for topic_id in topic_ids:
        t_dfs.append(corpus[corpus['topic_id'] == topic_id][start_topic:end_topic])
    small = pd.concat(t_dfs)
    return small

test_feeder_name = "bes_naive_datasplit_[0:5]"
test_feeder_descr = """
Using bert-extractive_summarizer with original naive datasplit.
Uses first 5 documents of each topic
Split entire topic into portions above 0.1 ratio, summarise iteratively
"""
test_feeder = SummaryFeeder(SummarizationHandler(sum_model), test_feeder_name, tech_descr=test_feeder_descr)

first_20 = small_corpus(0, 5)
test_feeder.summarize_topics(first_20)

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Summarising text in 1 pieces.


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:972: ConvergenceWarning: Number of distinct clusters (167) found smaller than n_clusters (205). Possibly due to duplicate points in X.
  return_n_iter=True)


100%|██████████| 1/1 [00:11<00:00, 11.12s/it]


  0%|          | 0/1 [00:00<?, ?it/s]

Summarising text in 1 pieces.




 11%|█         | 1/9 [00:17<02:21, 17.64s/it]

  0%|          | 0/1 [00:00<?, ?it/s]

...



 89%|████████▉ | 8/9 [12:42<01:39, 99.30s/it] 

  0%|          | 0/1 [00:00<?, ?it/s]

Summarising text in 1 pieces.


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:972: ConvergenceWarning: Number of distinct clusters (141) found smaller than n_clusters (180). Possibly due to duplicate points in X.
  return_n_iter=True)


100%|██████████| 1/1 [00:09<00:00,  9.45s/it]


  0%|          | 0/1 [00:00<?, ?it/s]

Summarising text in 1 pieces.




100%|██████████| 9/9 [12:57<00:00, 86.35s/it]


## Database Interaction

In [ ]:
# import sqlite3

# db_dir = '/nfs/proj-repo/AAARG-dissertation'
# db_name = 'sumresults.db'
# db_path = db_dir + '/' + db_name

# conn = sqlite3.connect(db_path)  # creates db if doesn't exist
# c = conn.cursor()  # allows send commands to db

In [ ]:
# c.execute("""CREATE TABLE results (
#     topic_id integer,
#     summary text
# )""")